# NLP модель сопоставления названий спортшкол
**Описание**

Сервис “Мой Чемпион” помогает спортивным школам фигурного катания, тренерам мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.

**Цель**

Создать решение для стандартизации названий спортивных школ.
Например одна и та же школа может быть записана по разному.


**Задачи**

Изучить данные – эталонные названия СШ и варианты пользовательского ввода. \
Подготовить обучающий набор данных на основе эталонного датасета.\
Создать модель для подбора наиболее вероятных названий при ошибочном вводе.\
Создать функцию (класс, модуль) для применения в сервисе.\
возможность выбора количества кандидатов.\
вывод в виде списка словарей.\
Протестировать решение.\
Проанализировать результат и предложить варианты улучшения.\
Создать документацию:
- описание признаков
- какая модель используется
- как оценивается качество
- инструкция по запуску (применению)

In [1]:
pip install nlpaug

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.2 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
# импортирую нужные библиотеки
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import nlpaug.augmenter.char as nac
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import semantic_search

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-07-01 16:58:13.950134: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-01 16:58:13.950260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-01 16:58:14.097287: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Загрузка и предобработка данных

In [4]:
# Путь файла
df_file = '/kaggle/input/dataframe/.csv'
reference_file = '/kaggle/input/reference/.csv'

# Загрузка данных
df = pd.read_csv(df_file)
reference = pd.read_csv(reference_file)

In [5]:
# Общая информация
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  895 non-null    int64 
 1   name       895 non-null    object
dtypes: int64(1), object(1)
memory usage: 14.1+ KB


,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""


In [6]:
# Общая информация
reference.info()
reference.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  306 non-null    int64 
 1   name       306 non-null    object
 2   region     306 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.3+ KB


,school_id,name,region
0,1,Авангард,Московская область
1,2,Авангард,Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан
3,4,Аврора,Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург


In [7]:
# Создам столбец с именем и регионом в датасете reference
reference['title'] = reference['name'] + ' ' + reference['region']
reference = reference.sort_values('region')
reference.sample(5)

,school_id,name,region,title
229,230,ФАУ МО РФ ЦСКА,Республика Татарстан,ФАУ МО РФ ЦСКА Республика Татарстан
139,140,РОО СФФК РСЯ,Республика Саха,РОО СФФК РСЯ Республика Саха
167,168,Старт,Пермский край,Старт Пермский край
277,280,Энергия льда,Санкт-Петербург,Энергия льда Санкт-Петербург
40,41,Гусева,Тверская область,Гусева Тверская область


In [8]:
# В эталонных названиях присутствуют неявные дубликаты, например:
reference.loc[(reference['region'] == 'Республика Корелия') | (reference['region'] == 'Костромская бласть')]

,school_id,name,region,title
38,39,Голубева,Костромская бласть,Голубева Костромская бласть
67,68,Керриган,Республика Корелия,Керриган Республика Корелия


Если покопаться, то можно обнаружить еще. Однако, это не относится к текущей задаче. Скорее тут как рекомендауцию, можно выделить - унификация процесса заполнения заявок, либо добавление еще признаков (наприммер, ИНН).

In [9]:
# Уберу лишнии символы - пробелы, кавычки и тд. Для этого создам функцию
def del_symbols(dataframe):
    for i in dataframe.columns:
        dataframe[i] = (dataframe[i].astype(str)
              .replace(r'[^А-Яа-яёЁA-Za-z0-9\s]', ' ', regex=True)
              .replace(r'\s+',' ',regex=True)
              .str.strip())

In [10]:
# Удаление символов из df
del_symbols(df)
df.head()

,school_id,name
0,1836,ООО Триумф
1,1836,Москва СК Триумф
2,610,СШОР Надежда Губернии
3,610,Саратовская область ГБУСО СШОР Надежда Губернии
4,609,СШ Гвоздика


In [11]:
# Удаление символов из reference
del_symbols(reference)
reference.sample(5)

,school_id,name,region,title
267,270,ЦФКСиЗ Невского района,Санкт Петербург,ЦФКСиЗ Невского района Санкт Петербург
144,145,РЦСПЗВС,Челябинская область,РЦСПЗВС Челябинская область
261,263,Юность,Мурманская область,Юность Мурманская область
76,77,Космос,Воронежская область,Космос Воронежская область
201,202,СШОР по ФФК,Республика Мордовия,СШОР по ФФК Республика Мордовия


In [12]:
# После обработки, посмотрим сколько в данных дубликатов. Поиск дубликатов веду по столбцу title, 
# ведь он содержит полную информацию
reference['title'].duplicated().sum()

1

In [13]:
# Удалю дубликаты
reference = reference.drop_duplicates('title')
reference['title'].duplicated().sum()

0

In [14]:
# Аналогично в df
print('Количество дубликатов до: ',df.duplicated().sum())
df = df.drop_duplicates()
print('Количество дубликатов после: ',df.duplicated().sum())

Количество дубликатов до:  90
Количество дубликатов после:  0


# Создание модели

Проведу аугментацию данных reference. Расширю датасет с 900 записей до примерно 3-х тысяч для лучшего обучения модели.

In [15]:
# Загрузка модуля аугментации
aug = nac.RandomCharAug()

# Аугментация
reference['aug'] = reference['title'].apply(lambda x: aug.augment(x, n=10))
reference.head()

,school_id,name,region,title,aug
301,305,Прогресс,Алтайский край,Прогресс Алтайский край,"[Прk#ресS Алтайский край, Gрог&xсс Алтайский к..."
21,22,Беломорец,Архангельская область,Беломорец Архангельская область,"[Беломорец Архангельская Jблpстc, Беломорец Ар..."
65,66,Каскад,Архангельская область,Каскад Архангельская область,"[Каскад Арpан^еR(ская область, Каскад Qрханr$л..."
158,159,Созвездие,Астраханская область,Созвездие Астраханская область,"[0озcезPие Астраханская область, Созвездие Аст..."
50,51,ДЮСШ по ЗВС,Белгородская область,ДЮСШ по ЗВС Белгородская область,"[vЮСM по ЗВС Белгородская Rбла7тj, ДЮСШ по ЗВС..."


In [16]:
# Оставлю только оригинальное название и аугментированные данные, разделю их и добавлю как новые строки
reference = reference.explode('aug')[['school_id','aug','title']].reset_index(drop=True)
reference.info()
reference.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3050 entries, 0 to 3049
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  3050 non-null   object
 1   aug        3050 non-null   object
 2   title      3050 non-null   object
dtypes: object(3)
memory usage: 71.6+ KB


,school_id,aug,title
0,305,Прk#ресS Алтайский край,Прогресс Алтайский край
1,305,Gрог&xсс Алтайский край,Прогресс Алтайский край
2,305,ПGогреkB Алтайский край,Прогресс Алтайский край
3,305,Прогресс Алтайский крzW,Прогресс Алтайский край
4,305,Прогресс Алтайский lраR,Прогресс Алтайский край


In [17]:
# Загрузка модели векторизации
model = SentenceTransformer('sentence-transformers/LaBSE')

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

In [18]:
corpus = model.encode(reference['title'].values)
corpus

Batches:   0%|          | 0/96 [00:00<?, ?it/s]

array([[ 0.02759258, -0.03165546,  0.00468466, ...,  0.0076037 ,
         0.02411924, -0.05793916],
       [ 0.02759258, -0.03165546,  0.00468466, ...,  0.0076037 ,
         0.02411924, -0.05793916],
       [ 0.02759258, -0.03165546,  0.00468466, ...,  0.0076037 ,
         0.02411924, -0.05793916],
       ...,
       [-0.0045743 , -0.04235094,  0.00392915, ..., -0.06342168,
         0.037862  , -0.02816418],
       [-0.0045743 , -0.04235094,  0.00392915, ..., -0.06342168,
         0.037862  , -0.02816418],
       [-0.0045743 , -0.04235094,  0.00392915, ..., -0.06342168,
         0.037862  , -0.02816418]], dtype=float32)

In [19]:
queries = model.encode(reference['aug'].values)
queries

Batches:   0%|          | 0/96 [00:00<?, ?it/s]

array([[ 0.04612003, -0.06084047,  0.00661312, ...,  0.01350922,
         0.04608053, -0.02592854],
       [ 0.05242554, -0.09849446,  0.00632825, ..., -0.01132567,
         0.02926574, -0.05118695],
       [ 0.04022485, -0.06398654,  0.0408589 , ...,  0.00150961,
         0.04884669, -0.05505911],
       ...,
       [-0.01379323, -0.07358073, -0.02445046, ..., -0.03059729,
         0.03981385, -0.01468448],
       [-0.0235868 , -0.05336544, -0.00719866, ..., -0.06797668,
         0.03273929,  0.02410294],
       [ 0.02648856, -0.05911833,  0.02244344, ..., -0.01325686,
         0.00310445,  0.04145979]], dtype=float32)

In [20]:
# Поиск выполню утилитой semantic_search
search_result = semantic_search(queries,corpus,top_k=5)
search_result

[[{'corpus_id': 0, 'score': 0.7001754641532898},
  {'corpus_id': 1, 'score': 0.7001754641532898},
  {'corpus_id': 3, 'score': 0.7001754641532898},
  {'corpus_id': 4, 'score': 0.7001754641532898},
  {'corpus_id': 2, 'score': 0.7001754641532898}],
 [{'corpus_id': 0, 'score': 0.66026771068573},
  {'corpus_id': 1, 'score': 0.66026771068573},
  {'corpus_id': 3, 'score': 0.66026771068573},
  {'corpus_id': 4, 'score': 0.66026771068573},
  {'corpus_id': 2, 'score': 0.66026771068573}],
 [{'corpus_id': 0, 'score': 0.7158888578414917},
  {'corpus_id': 1, 'score': 0.7158888578414917},
  {'corpus_id': 3, 'score': 0.7158888578414917},
  {'corpus_id': 4, 'score': 0.7158888578414917},
  {'corpus_id': 2, 'score': 0.7158888578414917}],
 [{'corpus_id': 0, 'score': 0.8830403089523315},
  {'corpus_id': 1, 'score': 0.8830403089523315},
  {'corpus_id': 3, 'score': 0.8830403089523315},
  {'corpus_id': 4, 'score': 0.8830403089523315},
  {'corpus_id': 2, 'score': 0.8830403089523315}],
 [{'corpus_id': 0, 'score'

In [21]:
#Вытащу найденный id и добавлю его в исходный датасет для сравнения
reference['candidate_idx'] = [x[0]['corpus_id'] for x in search_result]
reference['candidate_name'] = reference.title.values[reference.candidate_idx.values]
reference

,school_id,aug,title,candidate_idx,candidate_name
0,305,Прk#ресS Алтайский край,Прогресс Алтайский край,0,Прогресс Алтайский край
1,305,Gрог&xсс Алтайский край,Прогресс Алтайский край,0,Прогресс Алтайский край
2,305,ПGогреkB Алтайский край,Прогресс Алтайский край,0,Прогресс Алтайский край
3,305,Прогресс Алтайский крzW,Прогресс Алтайский край,0,Прогресс Алтайский край
4,305,Прогресс Алтайский lраR,Прогресс Алтайский край,0,Прогресс Алтайский край
...,...,...,...,...,...
3045,215,СШОР 4 Ярос^а63кrя __ласXь,СШОР 4 Ярославская область,3040,СШОР 4 Ярославская область
3046,215,СOОX 4 Tnос#авpкая область,СШОР 4 Ярославская область,840,УОР 1 Московская область
3047,215,СШОР 4 1роqлавсBfя о_)а%ть,СШОР 4 Ярославская область,3040,СШОР 4 Ярославская область
3048,215,СШОР 4 ЯрmJлbвс(ая EбVа6ть,СШОР 4 Ярославская область,3040,СШОР 4 Ярославская область


In [22]:
# Посчитаю точность модели
print('Точность модели: ',(reference['title']==reference['candidate_name']).sum()/len(reference))

Точность модели:  0.8613114754098361


In [23]:
# Проверю на тестовых данных - датасете df
queries_test = model.encode(df['name'].values)
search_test = semantic_search(queries_test,corpus,top_k=5)
search_test

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

[[{'corpus_id': 640, 'score': 0.742928683757782},
  {'corpus_id': 641, 'score': 0.742928683757782},
  {'corpus_id': 643, 'score': 0.742928683757782},
  {'corpus_id': 644, 'score': 0.742928683757782},
  {'corpus_id': 642, 'score': 0.742928683757782}],
 [{'corpus_id': 640, 'score': 0.8835117220878601},
  {'corpus_id': 641, 'score': 0.8835117220878601},
  {'corpus_id': 643, 'score': 0.8835117220878601},
  {'corpus_id': 644, 'score': 0.8835117220878601},
  {'corpus_id': 642, 'score': 0.8835117220878601}],
 [{'corpus_id': 2420, 'score': 0.7529895901679993},
  {'corpus_id': 2421, 'score': 0.7529895901679993},
  {'corpus_id': 2423, 'score': 0.7529895901679993},
  {'corpus_id': 2424, 'score': 0.7529895901679993},
  {'corpus_id': 2422, 'score': 0.7529895901679993}],
 [{'corpus_id': 2420, 'score': 0.9498438835144043},
  {'corpus_id': 2421, 'score': 0.9498438835144043},
  {'corpus_id': 2423, 'score': 0.9498438835144043},
  {'corpus_id': 2424, 'score': 0.9498438835144043},
  {'corpus_id': 2422, 's

In [24]:
df['candidate_idx'] = [x[0]['corpus_id'] for x in search_test]
df['candidate_name'] = reference.title.values[df.candidate_idx.values]
reference = reference.drop(['aug','candidate_idx','candidate_name'],axis=1).drop_duplicates()
df = df.merge(reference,left_on='candidate_name',right_on='title',how='left')
df.head()

,school_id_x,name,candidate_idx,candidate_name,school_id_y,title
0,1836,ООО Триумф,640,ООО Триумф Москва,1836,ООО Триумф Москва
1,1836,Москва СК Триумф,640,ООО Триумф Москва,1836,ООО Триумф Москва
2,610,СШОР Надежда Губернии,2420,СШОР Надежда Губернии Саратовская область,610,СШОР Надежда Губернии Саратовская область
3,610,Саратовская область ГБУСО СШОР Надежда Губернии,2420,СШОР Надежда Губернии Саратовская область,610,СШОР Надежда Губернии Саратовская область
4,609,СШ Гвоздика,2810,СШ Гвоздика Удмуртская республика,609,СШ Гвоздика Удмуртская республика


In [25]:
# Посчитаю точность модели
print('Точность модели на тесте: ',(df['school_id_y']==df['school_id_x']).sum()/len(df))

Точность модели на тесте:  0.6782608695652174


**Выводы**

Предоставленный эталонный датасет содержит некоторые неточности - разное наименование одних и тех же школ. Исходя из логики, большую уникальность имеет наименование региона. Названия школ труднее унифицировать - тут в дело могут идти многочисленные аббревиатуры, сокращения, кавычки и прочие символы. 

**Проделанная работа**

Была проведена подготовительная работа с данными - анализ и предобработка. Сделана аугментация эталонного датасета, чтобы более точно обучить модель. Модель векторизации данных - SentenceTransformer - LaBSE. Модель поиска сходства - semantic_search. Метрика accuracy на тренировочных данных 0.85, на тестовых - снизилась до 0.68.

**Рекомендации**

Из рекомендаций, можно уделить внимание именно исходным данным. В частности:
- Прочесать датасет и оставить только уникальные наименования. Все похожие удалить;
- Добавить другие уникальные признаки, например ИНН школы;
- Можно добавить кодификацию регионов, например по ОКТМО. Так как регион имеет большой вес внутри одного полного наименования - это может повысить точность распознавания;
- Если школ внутри региона много, можно попробовать распознавание по отдельным регионам. Так как регионы все таки изначально имеют большую универсальность и уникальность
- Изучить вопрос по введению единого шаблона для подачи заявок, например электронная заявка с выпадающим списком по регионам и названиям школ